In [1]:
# Kütüphaneleri yüklüyoruz.

import cv2
import time
import numpy as np

In [2]:
# Tracking işlemi yapıldığı için RAM dolup kod kapanmasın diye bunu çalıştırıyoruz.
# Bazı donanım ya da kütüphane sürümlerine gerekli olmayabilir.

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
# Burada modeli yüklüyoruz. 
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [5]:
font = cv2.FONT_HERSHEY_DUPLEX
kamera = cv2.VideoCapture("video2.mp4")



# Burada resmin boyutları 1280,720 
# Aşağıdaki 640 buradan geliyor. Yani ekranın yarısı. 660 ise çok az bir sağını ifade etmek için var.

region1=np.array([(640,0),(660,0),(660,720),(640,720)])
region1 = region1.reshape((-1,1,2))
total=set()

while True:
    
    

    ret, frame = kamera.read()
    
    if not ret:
        break
        
    H,W,_= frame.shape
    
    rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model.track(rgb_img, persist=True, verbose=False)
    labels=results[0].names
    
    
    
    
    center=int(W//2)
    # ekranı ikiye ayıran bir yatay çizgi. 
    
    cv2.line(frame, (center,0), (center,H), (0,0,255), 2)
    
    #cv2.polylines(frame,[region1],True,(255,0,0),2)
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        cls=results[0].boxes.cls[i]
        ids=results[0].boxes.id[i]
    
        x1,y1,x2,y2,score,cls,ids=int(x1),int(y1),int(x2),int(y2),float(score),int(cls),int(ids)
        name=labels[cls]
        
        if name!='bottle':
            continue
            
        if score<0.5:
            continue
            
        cx=int(x1/2+x2/2)
        cy=int(y1/2+y2/2)
        
        # Eğer orta noktanın x koordinatı bu değer arasındaysa ekranda gösterilsin diyoruz.
        # Yoksa koda bir etkisi yok
        if cx>620 and cx <670:
            cv2.circle(frame,(cx,cy),15,(255,0,0),-1)

        
      
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        if inside_region1>0:
            cv2.line(frame, (center,0), (center,H), (0,255,255), 2)
            total.add(ids)
        


    text='Total: '+str(len(total))
    frame[0:80,0:300]=(102,0,153)

    cv2.putText(frame, text,(0, 70), font, 2.2, (255,255,255), 2)

    
  
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
kamera.release()
cv2.destroyAllWindows()